In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/aitf/Amazon Sales for Modeling.csv'
df = pd.read_csv(path, delimiter=",")

print("Data berhasil dimuat!")

Data berhasil dimuat!


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183 entries, 0 to 1182
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   discounted_price            1183 non-null   float64
 1   actual_price                1183 non-null   float64
 2   discount_percentage         1183 non-null   float64
 3   rating                      1183 non-null   float64
 4   rating_count                1183 non-null   float64
 5   bayesian_score              1183 non-null   float64
 6   sentiment_score             1183 non-null   float64
 7   root_category_encoded       1183 non-null   int64  
 8   score_category_encoded      1183 non-null   int64  
 9   sentiment_category_encoded  1183 non-null   int64  
 10  score_category              1183 non-null   object 
 11  rooted_category             1183 non-null   object 
 12  cleaned_combined_review     1183 non-null   object 
 13  sentiment_category          1183 

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Preprocessing & feature engineering
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Model
from sklearn.linear_model import LogisticRegression

# Evaluation
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

In [ ]:
# Definisi Fitur dan Target
# Target
y = df['sentiment_category_encoded']

# Fitur
X = df[['cleaned_combined_review', 'bayesian_score', 'sentiment_score']]

# Train–Test Split (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Preprocessing dengan ColumnTransformer
# Kolom teks dan numerik
text_feature = 'cleaned_combined_review'
numeric_features = 'bayesian_score'

# Transformer
preprocessor = ColumnTransformer(
    transformers=[
        (
            'text',
            TfidfVectorizer(
                ngram_range=(1, 2),
                max_features=5000,
                min_df=5,
                stop_words='english'
            ),
            text_feature
        ),
        (
            'num',
            StandardScaler(),
            numeric_features
        )
    ]
)


In [ ]:
# Definisi Model
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    n_jobs=-1
)

# Pipeline End-to-End
pipeline = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
        ('classifier', model)
    ]
)

# Training Model
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_features=5000,
                                                                  min_df=5,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words='english'),
                                                  'cleaned_combined_review'),
                                                 ('num', StandardScaler(),
                                                  ['bayesian_score'])])),
                ('classifier',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    n_jobs=-1))])

In [ ]:
# Evaluasi pada Test Set
# Prediksi
y_pred = pipeline.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.5949

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.59      0.60        80
           1       0.48      0.55      0.51        80
           2       0.71      0.65      0.68        77

    accuracy                           0.59       237
   macro avg       0.61      0.60      0.60       237
weighted avg       0.60      0.59      0.60       237

Confusion Matrix:
[[47 28  5]
 [21 44 15]
 [ 8 19 50]]


In [ ]:
# Cross-Validation (Stratified K-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    pipeline,
    X,
    y,
    cv=cv,
    scoring='f1_weighted'
)

print("Cross-Validation F1-Weighted Scores:", cv_scores)
print("Mean CV F1-Weighted:", cv_scores.mean())


Cross-Validation F1-Weighted Scores: [0.62166552 0.57665611 0.57095501 0.66855716 0.58651834]
Mean CV F1-Weighted: 0.6048704278066158


In [ ]:
# Ambil TF-IDF dan koefisien
tfidf = pipeline.named_steps['preprocessing'].named_transformers_['text']
feature_names = tfidf.get_feature_names_out()

coefficients = pipeline.named_steps['classifier'].coef_

# Contoh: 10 kata paling berpengaruh untuk tiap kelas
for idx, class_coef in enumerate(coefficients):
    # Only consider coefficients corresponding to text features
    text_coefs = class_coef[:len(feature_names)]
    top_features_indices = np.argsort(text_coefs)[-10:]
    print(f"\nTop words for class {idx}:")
    print(feature_names[top_features_indices])


Top words for class 0:
['broken' 'nice good' 'gud' 'batteries' 'connecting' 'ok' 'worst' 'poor'
 'working' 'bad']

Top words for class 1:
['value' 'money' 'use' 'useful' 'good quality' 'quality' 'holder' 'coffee'
 'good product' 'good']

Top words for class 2:
['boat' 'brightness' 'screen' 'best' 'watch' 'easy' 'sound' 'recommend'
 'bass' 'great']


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definisi Model Baru: RandomForestClassifier
new_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    random_state=42,   # For reproducibility
    n_jobs=-1          # Use all available cores
)

# Pipeline End-to-End dengan model baru
new_pipeline = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
        ('classifier', new_model)
    ]
)

print("New model (RandomForestClassifier) and pipeline defined.")

New model (RandomForestClassifier) and pipeline defined.


In [ ]:
# Training Model Baru
print("Training new model...")
new_pipeline.fit(X_train, y_train)
print("New model training complete!")

Training new model...
New model training complete!


In [ ]:
# Evaluasi Model Baru pada Test Set
# Prediksi
new_y_pred = new_pipeline.predict(X_test)

# Accuracy
new_accuracy = accuracy_score(y_test, new_y_pred)
print(f"\nNew Model (RandomForestClassifier) Accuracy: {new_accuracy:.4f}")

# Classification Report
print("\nNew Model (RandomForestClassifier) Classification Report:")
print(classification_report(y_test, new_y_pred))

# Confusion Matrix
print("New Model (RandomForestClassifier) Confusion Matrix:")
print(confusion_matrix(y_test, new_y_pred))


New Model (RandomForestClassifier) Accuracy: 0.6582

New Model (RandomForestClassifier) Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.70      0.69        80
           1       0.55      0.57      0.56        80
           2       0.77      0.70      0.73        77

    accuracy                           0.66       237
   macro avg       0.66      0.66      0.66       237
weighted avg       0.66      0.66      0.66       237

New Model (RandomForestClassifier) Confusion Matrix:
[[56 20  4]
 [22 46 12]
 [ 5 18 54]]


In [ ]:
# Cross-Validation (Stratified K-Fold) untuk Model Baru
new_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

new_cv_scores = cross_val_score(
    new_pipeline,
    X,
    y,
    cv=new_cv,
    scoring='f1_weighted' # Consistent scoring with previous model
)

print("\nNew Model (RandomForestClassifier) Cross-Validation F1-Weighted Scores:", new_cv_scores)
print("New Model (RandomForestClassifier) Mean CV F1-Weighted:", new_cv_scores.mean())


New Model (RandomForestClassifier) Cross-Validation F1-Weighted Scores: [0.68375506 0.62226284 0.63351919 0.62978512 0.55321901]
New Model (RandomForestClassifier) Mean CV F1-Weighted: 0.6245082432011012


In [ ]:
# Definisi Fitur untuk model hanya dengan fitur numerik
X_numeric_only = df[numeric_features]
y_numeric_only = df['sentiment_category_encoded']

# Train–Test Split (Stratified) untuk fitur numerik saja
X_train_numeric, X_test_numeric, y_train_numeric, y_test_numeric = train_test_split(
    X_numeric_only,
    y_numeric_only,
    test_size=0.2,
    random_state=42,
    stratify=y_numeric_only
)

# Preprocessing dengan ColumnTransformer hanya untuk fitur numerik
numeric_only_preprocessor = ColumnTransformer(
    transformers=[
        (
            'num',
            StandardScaler(),
            numeric_features
        )
    ]
)

print("Numeric-only feature set and preprocessor defined.")

Numeric-only feature set and preprocessor defined.


In [ ]:
# Definisi Model Baru: RandomForestClassifier (hanya numerik)
numeric_rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# Pipeline End-to-End dengan model baru dan preprocessor numerik
numeric_rf_pipeline = Pipeline(
    steps=[
        ('preprocessing', numeric_only_preprocessor),
        ('classifier', numeric_rf_model)
    ]
)

print("Numeric-only RandomForestClassifier pipeline defined.")

Numeric-only RandomForestClassifier pipeline defined.


In [ ]:
# Training Model RandomForest (hanya numerik)
print("Training numeric-only RandomForest model...")
numeric_rf_pipeline.fit(X_train_numeric, y_train_numeric)
print("Numeric-only RandomForest model training complete!")

Training numeric-only RandomForest model...
Numeric-only RandomForest model training complete!


In [ ]:
# Evaluasi Model RandomForest (hanya numerik) pada Test Set
# Prediksi
numeric_rf_y_pred = numeric_rf_pipeline.predict(X_test_numeric)

# Accuracy
numeric_rf_accuracy = accuracy_score(y_test_numeric, numeric_rf_y_pred)
print(f"\nNumeric-only RandomForestClassifier Accuracy: {numeric_rf_accuracy:.4f}")

# Classification Report
print("\nNumeric-only RandomForestClassifier Classification Report:")
print(classification_report(y_test_numeric, numeric_rf_y_pred))

# Confusion Matrix
print("Numeric-only RandomForestClassifier Confusion Matrix:")
print(confusion_matrix(y_test_numeric, numeric_rf_y_pred))


Numeric-only RandomForestClassifier Accuracy: 0.3755

Numeric-only RandomForestClassifier Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.31      0.34        80
           1       0.37      0.41      0.39        80
           2       0.39      0.40      0.39        77

    accuracy                           0.38       237
   macro avg       0.38      0.38      0.37       237
weighted avg       0.38      0.38      0.37       237

Numeric-only RandomForestClassifier Confusion Matrix:
[[25 34 21]
 [19 33 28]
 [23 23 31]]


In [ ]:
# Cross-Validation (Stratified K-Fold) untuk Model RandomForest (hanya numerik)
numeric_rf_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

numeric_rf_cv_scores = cross_val_score(
    numeric_rf_pipeline,
    X_numeric_only,
    y_numeric_only,
    cv=numeric_rf_cv,
    scoring='f1_weighted'
)

print("\nNumeric-only RandomForestClassifier Cross-Validation F1-Weighted Scores:", numeric_rf_cv_scores)
print("Numeric-only RandomForestClassifier Mean CV F1-Weighted:", numeric_rf_cv_scores.mean())


Numeric-only RandomForestClassifier Cross-Validation F1-Weighted Scores: [0.34944902 0.32353441 0.35826339 0.35113448 0.40001524]
Numeric-only RandomForestClassifier Mean CV F1-Weighted: 0.35647930757681273


# Task
Define a new ColumnTransformer that applies TF-IDF vectorization followed by TruncatedSVD for the 'cleaned_combined_review' text feature, and StandardScaler for the numeric features ('bayesian_score', 'sentiment_score'). Then, initialize a RandomForestClassifier model and create a new end-to-end pipeline that combines this preprocessor with TruncatedSVD and the RandomForestClassifier. Train this new pipeline using the training data (X_train, y_train) and evaluate its performance on the test set, including accuracy, classification report, and confusion matrix. Perform stratified K-fold cross-validation on this model using the entire dataset (X, y) and report the F1-weighted scores and their mean. Finally, provide a comprehensive comparison of all trained models (Logistic Regression (Text+Numeric), RandomForestClassifier (Text+Numeric), RandomForestClassifier (Numeric Only), and the new RandomForestClassifier (Text+SVD+Numeric)), focusing on key metrics such as accuracy, F1-macro, F1-weighted, and cross-validation scores, and summarize the findings.

## Define Preprocessor with TruncatedSVD

### Subtask:
Define a new ColumnTransformer that applies TF-IDF vectorization followed by TruncatedSVD for the 'cleaned_combined_review' text feature, and StandardScaler for the numeric features ('bayesian_score', 'sentiment_score').


**Reasoning**:
First, I'll import TruncatedSVD from `sklearn.decomposition`, which is necessary for dimensionality reduction on the TF-IDF features. Then, I'll define a pipeline for the text features that applies TF-IDF vectorization followed by TruncatedSVD. Finally, I'll create a ColumnTransformer to handle both text and numeric features with their respective transformers and print a confirmation message.



In [ ]:
from sklearn.decomposition import TruncatedSVD

# Pipeline for text feature (TF-IDF + TruncatedSVD)
text_transformer_svd = Pipeline(
    steps=[
        (
            'tfidf',
            TfidfVectorizer(
                ngram_range=(1, 2),
                max_features=5000,
                min_df=5,
                stop_words='english'
            )
        ),
        (
            'svd',
            TruncatedSVD(n_components=200, random_state=42)
        )
    ]
)

# Define new ColumnTransformer with TruncatedSVD for text features
svd_preprocessor = ColumnTransformer(
    transformers=[
        (
            'text_svd',
            text_transformer_svd,
            text_feature
        ),
        (
            'num',
            StandardScaler(),
            numeric_features
        )
    ]
)

print("SVD preprocessor defined with TF-IDF and TruncatedSVD for text features.")

SVD preprocessor defined with TF-IDF and TruncatedSVD for text features.


**Reasoning**:
Now that the `svd_preprocessor` is defined, the next step is to initialize a `RandomForestClassifier` model and then combine this preprocessor with the new model into an end-to-end pipeline, as specified in the subtask instructions.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definisi Model Baru: RandomForestClassifier
svd_rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees in the forest
    random_state=42,   # For reproducibility
    n_jobs=-1          # Use all available cores
)

# Pipeline End-to-End dengan model baru dan preprocessor SVD
svd_rf_pipeline = Pipeline(
    steps=[
        ('preprocessing', svd_preprocessor),
        ('classifier', svd_rf_model)
    ]
)

print("RandomForestClassifier model with SVD preprocessor and pipeline defined.")

RandomForestClassifier model with SVD preprocessor and pipeline defined.


**Reasoning**:
Now that the new pipeline (`svd_rf_pipeline`) with the SVD preprocessor and RandomForestClassifier is defined, I will train this pipeline using the training data (`X_train`, `y_train`) as per the instructions.



In [ ]:
print("Training new RandomForestClassifier model with SVD preprocessor...")
svd_rf_pipeline.fit(X_train, y_train)
print("New RandomForestClassifier model with SVD preprocessor training complete!")

Training new RandomForestClassifier model with SVD preprocessor...
New RandomForestClassifier model with SVD preprocessor training complete!


**Reasoning**:
Now that the new pipeline (`svd_rf_pipeline`) has been trained, I will evaluate its performance on the test set by making predictions and calculating accuracy, generating a classification report, and a confusion matrix, as specified in the main task.



In [ ]:
print("\nEvaluating New Model (RandomForestClassifier with SVD) on Test Set...")
# Prediksi
svd_rf_y_pred = svd_rf_pipeline.predict(X_test)

# Accuracy
svd_rf_accuracy = accuracy_score(y_test, svd_rf_y_pred)
print(f"New Model (RandomForestClassifier with SVD) Accuracy: {svd_rf_accuracy:.4f}")

# Classification Report
print("\nNew Model (RandomForestClassifier with SVD) Classification Report:")
print(classification_report(y_test, svd_rf_y_pred))

# Confusion Matrix
print("New Model (RandomForestClassifier with SVD) Confusion Matrix:")
print(confusion_matrix(y_test, svd_rf_y_pred))


Evaluating New Model (RandomForestClassifier with SVD) on Test Set...
New Model (RandomForestClassifier with SVD) Accuracy: 0.5527

New Model (RandomForestClassifier with SVD) Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.36      0.46        80
           1       0.49      0.53      0.51        80
           2       0.57      0.78      0.66        77

    accuracy                           0.55       237
   macro avg       0.56      0.56      0.54       237
weighted avg       0.56      0.55      0.54       237

New Model (RandomForestClassifier with SVD) Confusion Matrix:
[[29 30 21]
 [13 42 25]
 [ 4 13 60]]


**Reasoning**:
After evaluating the new model's performance on the test set, the next step is to perform stratified K-fold cross-validation using the entire dataset (X, y) to get a more robust estimate of its performance, as specified in the main task.



In [ ]:
print("\nPerforming Cross-Validation for New Model (RandomForestClassifier with SVD)...")
# Cross-Validation (Stratified K-Fold) untuk Model Baru dengan SVD
svd_rf_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

svd_rf_cv_scores = cross_val_score(
    svd_rf_pipeline,
    X,
    y,
    cv=svd_rf_cv,
    scoring='f1_weighted'
)

print("New Model (RandomForestClassifier with SVD) Cross-Validation F1-Weighted Scores:", svd_rf_cv_scores)
print("New Model (RandomForestClassifier with SVD) Mean CV F1-Weighted:", svd_rf_cv_scores.mean())


Performing Cross-Validation for New Model (RandomForestClassifier with SVD)...
New Model (RandomForestClassifier with SVD) Cross-Validation F1-Weighted Scores: [0.53807077 0.52359554 0.52871039 0.57948012 0.5228721 ]
New Model (RandomForestClassifier with SVD) Mean CV F1-Weighted: 0.5385457840304557


## Model Comparison and Summary

To provide a comprehensive comparison, let's collect the key performance metrics for all the models we've trained:

1.  **Logistic Regression (Text + Numeric Features)**
2.  **RandomForestClassifier (Text + Numeric Features)**
3.  **RandomForestClassifier (Numeric Only Features)**
4.  **RandomForestClassifier (Text (TF-IDF + SVD) + Numeric Features)**

We will compare them based on accuracy, F1-macro, F1-weighted (from test set evaluation), and mean F1-weighted from cross-validation.

**Reasoning**:
To systematically compare the models, I need to extract the relevant metrics (accuracy, F1-macro, F1-weighted, and mean CV F1-weighted) from the execution outputs for each model and consolidate them into a structured format like a Pandas DataFrame. This will involve parsing the classification report strings to get F1 scores.



In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

# --- Metrics for Logistic Regression (Text+Numeric) ---
logreg_accuracy = accuracy
logreg_cr_str = """
              precision    recall  f1-score   support

           0       0.83      0.65      0.73        80
           1       0.53      0.64      0.58        80
           2       0.71      0.71      0.71        77

    accuracy                           0.67       237
   macro avg       0.69      0.67      0.67       237
weighted avg       0.69      0.67      0.67       237
"""

# Helper function to parse classification report string
def parse_cr(cr_str):
    lines = cr_str.strip().split('\n')
    macro_line = [line for line in lines if 'macro avg' in line][0]
    weighted_line = [line for line in lines if 'weighted avg' in line][0]

    macro_f1 = float(macro_line.split()[3])
    weighted_f1 = float(weighted_line.split()[3])
    return macro_f1, weighted_f1

logreg_macro_f1, logreg_weighted_f1 = parse_cr(logreg_cr_str)
logreg_mean_cv_f1 = cv_scores.mean()

# --- Metrics for RandomForestClassifier (Text+Numeric) ---
rf_text_numeric_accuracy = new_accuracy
rf_text_numeric_cr_str = """
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        80
           1       0.83      0.80      0.82        80
           2       0.82      0.86      0.84        77

    accuracy                           0.87       237
   macro avg       0.87      0.87      0.87       237
weighted avg       0.87      0.87      0.87       237
"""
rf_text_numeric_macro_f1, rf_text_numeric_weighted_f1 = parse_cr(rf_text_numeric_cr_str)
rf_text_numeric_mean_cv_f1 = new_cv_scores.mean()

# --- Metrics for RandomForestClassifier (Numeric Only) ---
rf_numeric_only_accuracy = numeric_rf_accuracy
rf_numeric_only_cr_str = """
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00        80
           2       1.00      1.00      1.00        77

    accuracy                           1.00       237
   macro avg       1.00      1.00      1.00       237
weighted avg       1.00      1.00      1.00       237
"""
rf_numeric_only_macro_f1, rf_numeric_only_weighted_f1 = parse_cr(rf_numeric_only_cr_str)
rf_numeric_only_mean_cv_f1 = numeric_rf_cv_scores.mean()

# --- Metrics for RandomForestClassifier (Text+SVD+Numeric) ---
rf_svd_accuracy = svd_rf_accuracy
rf_svd_cr_str = """
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        80
           1       0.99      0.96      0.97        80
           2       0.97      0.99      0.98        77

    accuracy                           0.98       237
   macro avg       0.98      0.98      0.98       237
weighted avg       0.98      0.98      0.98       237
"""
rf_svd_macro_f1, rf_svd_weighted_f1 = parse_cr(rf_svd_cr_str)
rf_svd_mean_cv_f1 = svd_rf_cv_scores.mean()

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Model': [
        'Logistic Regression (Text+Numeric)',
        'RandomForestClassifier (Text+Numeric)',
        'RandomForestClassifier (Numeric Only)',
        'RandomForestClassifier (Text+SVD+Numeric)'
    ],
    'Test Accuracy': [
        logreg_accuracy,
        rf_text_numeric_accuracy,
        rf_numeric_only_accuracy,
        rf_svd_accuracy
    ],
    'Test F1-Macro': [
        logreg_macro_f1,
        rf_text_numeric_macro_f1,
        rf_numeric_only_macro_f1,
        rf_svd_macro_f1
    ],
    'Test F1-Weighted': [
        logreg_weighted_f1,
        rf_text_numeric_weighted_f1,
        rf_numeric_only_weighted_f1,
        rf_svd_weighted_f1
    ],
    'Mean CV F1-Weighted': [
        logreg_mean_cv_f1,
        rf_text_numeric_mean_cv_f1,
        rf_numeric_only_mean_cv_f1,
        rf_svd_mean_cv_f1
    ]
})

print("Model Performance Comparison:")
print(comparison_df.round(4).to_markdown(index=False))


Model Performance Comparison:
| Model                                     |   Test Accuracy |   Test F1-Macro |   Test F1-Weighted |   Mean CV F1-Weighted |
|:------------------------------------------|----------------:|----------------:|-------------------:|----------------------:|
| Logistic Regression (Text+Numeric)        |          0.5949 |            0.67 |               0.67 |                0.6049 |
| RandomForestClassifier (Text+Numeric)     |          0.6582 |            0.87 |               0.87 |                0.6245 |
| RandomForestClassifier (Numeric Only)     |          0.3755 |            1    |               1    |                0.3565 |
| RandomForestClassifier (Text+SVD+Numeric) |          0.5527 |            0.98 |               0.98 |                0.5385 |


## Final Task

### Subtask:
Summarize the findings and the performance of all models.


## Summary:

### Q&A
The analysis successfully summarized the findings and performance of all models by comparing their key metrics: Test Accuracy, Test F1-Macro, Test F1-Weighted, and Mean CV F1-Weighted. The comparison revealed that the RandomForestClassifier using only numeric features achieved the highest performance across all metrics, closely followed by the RandomForestClassifier incorporating text features preprocessed with TF-IDF and TruncatedSVD along with numeric features.

### Data Analysis Key Findings
*   A new `ColumnTransformer` was defined, applying TF-IDF vectorization followed by `TruncatedSVD` for text features and `StandardScaler` for numeric features.
*   A `RandomForestClassifier` model was initialized and integrated into an end-to-end pipeline with the new preprocessor.
*   The newly trained `RandomForestClassifier` (Text (TF-IDF + SVD) + Numeric) achieved a test accuracy of $0.9831$, a test F1-weighted score of $0.98$, and a mean cross-validation F1-weighted score of $0.9849$.
*   A comprehensive comparison of all trained models yielded the following performance metrics:

| Model                                     | Test Accuracy | Test F1-Macro | Test F1-Weighted | Mean CV F1-Weighted |
| :---------------------------------------- | :------------ | :------------ | :--------------- | :------------------ |
| Logistic Regression (Text+Numeric)        | $0.6667$      | $0.67$        | $0.67$           | $0.6986$            |
| RandomForestClassifier (Text+Numeric)     | $0.8692$      | $0.87$        | $0.87$           | $0.8574$            |
| RandomForestClassifier (Numeric Only)     | $1.0000$      | $1.00$        | $1.00$           | $0.9992$            |
| RandomForestClassifier (Text+SVD+Numeric) | $0.9831$      | $0.98$        | $0.98$           | $0.9849$            |

*   The `RandomForestClassifier (Numeric Only)` model demonstrated the highest performance, achieving perfect scores in test accuracy, F1-macro, and F1-weighted, with a mean CV F1-weighted of $0.9992$.
*   The `RandomForestClassifier (Text+SVD+Numeric)` model showed a significant improvement over the `RandomForestClassifier (Text+Numeric)` model (which used raw TF-IDF features), indicating the effectiveness of `TruncatedSVD` in dimension reduction for text features.

### Insights or Next Steps
*   The numeric features ('bayesian_score', 'sentiment_score') are exceptionally strong predictors in this dataset, to the extent that a model trained solely on them achieved near-perfect performance.
*   Further investigation into the `RandomForestClassifier (Numeric Only)` model's perfect performance on the test set is warranted to confirm robustness and avoid potential data leakage or overfitting, although cross-validation scores also indicate very high performance.


# Task
Recalculate and display the comparison table with the correct accuracy, F1-macro, F1-weighted, and mean CV F1-weighted scores for all trained models, then summarize the findings and performance of all models based on this updated comparison, highlighting any changes due to the numeric features modification.

## Recalculate and Display Comparison Table

### Subtask:
Extract the correct accuracy, F1-macro, F1-weighted, and mean CV F1-weighted scores for all trained models directly from their respective execution outputs. Consolidate these updated metrics into a new Pandas DataFrame and display it, ensuring the comparison accurately reflects the current state of the models.


**Reasoning**:
To provide an accurate comparison, I will update the classification report strings, accuracy values, and mean cross-validation F1-weighted scores for all models with their correct outputs from the executed cells. Then, I will re-execute the code to generate the updated comparison DataFrame and display it.



In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

# --- Metrics for Logistic Regression (Text+Numeric) ---
logreg_accuracy = accuracy
logreg_cr_str = """
              precision    recall  f1-score   support

           0       0.62      0.59      0.60        80
           1       0.48      0.55      0.51        80
           2       0.71      0.65      0.68        77

    accuracy                           0.59       237
   macro avg       0.61      0.60      0.60       237
weighted avg       0.60      0.59      0.60       237
"""

# Helper function to parse classification report string
def parse_cr(cr_str):
    lines = cr_str.strip().split('\n')
    macro_line = [line for line in lines if 'macro avg' in line][0]
    weighted_line = [line for line in lines if 'weighted avg' in line][0]

    macro_f1 = float(macro_line.split()[3])
    weighted_f1 = float(weighted_line.split()[3])
    return macro_f1, weighted_f1

logreg_macro_f1, logreg_weighted_f1 = parse_cr(logreg_cr_str)
logreg_mean_cv_f1 = cv_scores.mean()

# --- Metrics for RandomForestClassifier (Text+Numeric) ---
rf_text_numeric_accuracy = new_accuracy
rf_text_numeric_cr_str = """
              precision    recall  f1-score   support

           0       0.67      0.70      0.69        80
           1       0.55      0.57      0.56        80
           2       0.77      0.70      0.73        77

    accuracy                           0.66       237
   macro avg       0.66      0.66      0.66       237
weighted avg       0.66      0.66      0.66       237
"""
rf_text_numeric_macro_f1, rf_text_numeric_weighted_f1 = parse_cr(rf_text_numeric_cr_str)
rf_text_numeric_mean_cv_f1 = new_cv_scores.mean()

# --- Metrics for RandomForestClassifier (Numeric Only) ---
rf_numeric_only_accuracy = numeric_rf_accuracy
rf_numeric_only_cr_str = """
              precision    recall  f1-score   support

           0       0.37      0.31      0.34        80
           1       0.37      0.41      0.39        80
           2       0.39      0.40      0.39        77

    accuracy                           0.38       237
   macro avg       0.38      0.38      0.37       237
weighted avg       0.38      0.38      0.37       237
"""
rf_numeric_only_macro_f1, rf_numeric_only_weighted_f1 = parse_cr(rf_numeric_only_cr_str)
rf_numeric_only_mean_cv_f1 = numeric_rf_cv_scores.mean()

# --- Metrics for RandomForestClassifier (Text+SVD+Numeric) ---
rf_svd_accuracy = svd_rf_accuracy
rf_svd_cr_str = """
              precision    recall  f1-score   support

           0       0.63      0.36      0.46        80
           1       0.49      0.53      0.51        80
           2       0.57      0.78      0.66        77

    accuracy                           0.55       237
   macro avg       0.56      0.56      0.54       237
weighted avg       0.56      0.55      0.54       237
"""
rf_svd_macro_f1, rf_svd_weighted_f1 = parse_cr(rf_svd_cr_str)
rf_svd_mean_cv_f1 = svd_rf_cv_scores.mean()

# --- Metrics for LinearSVC (Text+Numeric) ---
lsvm_accuracy = lsvm_accuracy
lsvm_cr_str = """
              precision    recall  f1-score   support

           0       0.64      0.59      0.61        80
           1       0.54      0.51      0.53        80
           2       0.68      0.78      0.73        77

    accuracy                           0.62       237
   macro avg       0.62      0.63      0.62       237
weighted avg       0.62      0.62      0.62       237
"""
lsvm_macro_f1, lsvm_weighted_f1 = parse_cr(lsvm_cr_str)
lsvm_mean_cv_f1 = lsvm_cv_scores.mean()

# --- Metrics for Multinomial Naïve Bayes (Text+Numeric) ---
mnb_accuracy = mnb_accuracy
mnb_cr_str = """
              precision    recall  f1-score   support

           0       0.76      0.35      0.48        80
           1       0.46      0.72      0.56        80
           2       0.64      0.61      0.62        77

    accuracy                           0.56       237
   macro avg       0.62      0.56      0.55       237
weighted avg       0.62      0.56      0.55       237
"""
mnb_macro_f1, mnb_weighted_f1 = parse_cr(mnb_cr_str)
mnb_mean_cv_f1 = mnb_cv_scores.mean()

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Model': [
        'Logistic Regression (Text+Numeric)',
        'RandomForestClassifier (Text+Numeric)',
        'RandomForestClassifier (Numeric Only)',
        'RandomForestClassifier (Text+SVD+Numeric)',
        'LinearSVC (Text+Numeric)',
        'Multinomial Naïve Bayes (Text+Numeric)'
    ],
    'Test Accuracy': [
        logreg_accuracy,
        rf_text_numeric_accuracy,
        rf_numeric_only_accuracy,
        rf_svd_accuracy,
        lsvm_accuracy,
        mnb_accuracy
    ],
    'Test F1-Macro': [
        logreg_macro_f1,
        rf_text_numeric_macro_f1,
        rf_numeric_only_macro_f1,
        rf_svd_macro_f1,
        lsvm_macro_f1,
        mnb_macro_f1
    ],
    'Test F1-Weighted': [
        logreg_weighted_f1,
        rf_text_numeric_weighted_f1,
        rf_numeric_only_weighted_f1,
        rf_svd_weighted_f1,
        lsvm_weighted_f1,
        mnb_weighted_f1
    ],
    'Mean CV F1-Weighted': [
        logreg_mean_cv_f1,
        rf_text_numeric_mean_cv_f1,
        rf_numeric_only_mean_cv_f1,
        rf_svd_mean_cv_f1,
        lsvm_mean_cv_f1,
        mnb_mean_cv_f1
    ]
})

print("Model Performance Comparison:")
print(comparison_df.round(4).to_markdown(index=False))

Model Performance Comparison:
| Model                                     |   Test Accuracy |   Test F1-Macro |   Test F1-Weighted |   Mean CV F1-Weighted |
|:------------------------------------------|----------------:|----------------:|-------------------:|----------------------:|
| Logistic Regression (Text+Numeric)        |          0.5949 |            0.6  |               0.59 |                0.6049 |
| RandomForestClassifier (Text+Numeric)     |          0.6582 |            0.66 |               0.66 |                0.6245 |
| RandomForestClassifier (Numeric Only)     |          0.3755 |            0.38 |               0.38 |                0.3565 |
| RandomForestClassifier (Text+SVD+Numeric) |          0.5527 |            0.56 |               0.55 |                0.5385 |
| LinearSVC (Text+Numeric)                  |          0.6245 |            0.63 |               0.62 |                0.5893 |
| Multinomial Naïve Bayes (Text+Numeric)    |          0.5612 |            0.56 |

## Summary:

### Q&A
The analysis successfully summarized the findings and performance of all models by comparing their key metrics: Test Accuracy, Test F1-Macro, Test F1-Weighted, and Mean CV F1-Weighted. The comparison revealed that the `RandomForestClassifier (Text+Numeric)` model achieved the highest performance among the models using both text and numeric features. The `RandomForestClassifier (Numeric Only)` model showed the lowest performance, indicating that numeric features alone are not sufficient for good classification in this dataset.

### Data Analysis Key Findings
*   A new `ColumnTransformer` was defined, applying TF-IDF vectorization followed by `TruncatedSVD` for text features and `StandardScaler` for numeric features.
*   A `RandomForestClassifier` model was initialized and integrated into an end-to-end pipeline with the new preprocessor.
*   The `RandomForestClassifier (Text (TF-IDF + SVD) + Numeric)` model achieved a test accuracy of `0.5527`, a test F1-weighted score of `0.55`, and a mean cross-validation F1-weighted score of `0.5385`.
*   A comprehensive comparison of all trained models yielded the following updated performance metrics:

| Model                                     | Test Accuracy | Test F1-Macro | Test F1-Weighted | Mean CV F1-Weighted |
| :---------------------------------------- | :------------ | :------------ | :--------------- | :------------------ |
| Logistic Regression (Text+Numeric)        | 0.5949        | 0.60          | 0.59             | 0.6049              |
| RandomForestClassifier (Text+Numeric)     | 0.6582        | 0.66          | 0.66             | 0.6245              |
| RandomForestClassifier (Numeric Only)     | 0.3755        | 0.38          | 0.38             | 0.3565              |
| RandomForestClassifier (Text+SVD+Numeric) | 0.5527        | 0.56          | 0.55             | 0.5385              |

*   The `RandomForestClassifier (Text+Numeric)` model demonstrated the best overall performance, with the highest test accuracy (0.6582), F1-macro (0.66), F1-weighted (0.66), and mean CV F1-weighted (0.6245) scores among all models.
*   The `RandomForestClassifier (Numeric Only)` model performed poorly, suggesting that text features are crucial for this classification task.
*   The `RandomForestClassifier (Text+SVD+Numeric)` model performed worse than the `RandomForestClassifier (Text+Numeric)` (which used raw TF-IDF features), indicating that `TruncatedSVD` with `n_components=200` might have removed too much information or that the original TF-IDF features were more effective for this dataset.
*   Logistic Regression provided competitive results, particularly in cross-validation, but was slightly outperformed by the RandomForestClassifier with full text and numeric features.

### Insights or Next Steps
*   The `cleaned_combined_review` text feature is a significant contributor to model performance, as evidenced by the poor performance of the numeric-only model.
*   `TruncatedSVD` for dimensionality reduction on TF-IDF features did not improve performance in this case. Further tuning of `n_components` for TruncatedSVD or exploring other dimensionality reduction techniques could be considered, or simply sticking with the full TF-IDF features if computational resources allow.
*   The `RandomForestClassifier` appears to be a stronger model for this task compared to `Logistic Regression` when text features are included.
*   Further hyperparameter tuning for the `RandomForestClassifier` (e.g., `max_depth`, `min_samples_split`, `min_samples_leaf`) could potentially yield even better results.

## Summary:

### Q&A
The analysis successfully summarized the findings and performance of all models by comparing their key metrics: Test Accuracy, Test F1-Macro, Test F1-Weighted, and Mean CV F1-Weighted. The comparison revealed that the `RandomForestClassifier (Text+Numeric)` model achieved the highest performance among the models using both text and numeric features. The `RandomForestClassifier (Numeric Only)` model showed the lowest performance, indicating that numeric features alone are not sufficient for good classification in this dataset.

### Data Analysis Key Findings
*   A new `ColumnTransformer` was defined, applying TF-IDF vectorization followed by `TruncatedSVD` for text features and `StandardScaler` for numeric features.
*   A `RandomForestClassifier` model was initialized and integrated into an end-to-end pipeline with the new preprocessor.
*   The `RandomForestClassifier (Text (TF-IDF + SVD) + Numeric)` model achieved a test accuracy of `0.5527`, a test F1-weighted score of `0.55`, and a mean cross-validation F1-weighted score of `0.5385`.
*   A comprehensive comparison of all trained models yielded the following updated performance metrics:

| Model                                     | Test Accuracy | Test F1-Macro | Test F1-Weighted | Mean CV F1-Weighted |
| :---------------------------------------- | :------------ | :------------ | :--------------- | :------------------ |
| Logistic Regression (Text+Numeric)        | 0.5949        | 0.60          | 0.59             | 0.6049              |
| RandomForestClassifier (Text+Numeric)     | 0.6582        | 0.66          | 0.66             | 0.6245              |
| RandomForestClassifier (Numeric Only)     | 0.3755        | 0.38          | 0.38             | 0.3565              |
| RandomForestClassifier (Text+SVD+Numeric) | 0.5527        | 0.56          | 0.55             | 0.5385              |

*   The `RandomForestClassifier (Text+Numeric)` model demonstrated the best overall performance, with the highest test accuracy (0.6582), F1-macro (0.66), F1-weighted (0.66), and mean CV F1-weighted (0.6245) scores among all models.
*   The `RandomForestClassifier (Numeric Only)` model performed poorly, suggesting that text features are crucial for this classification task.
*   The `RandomForestClassifier (Text+SVD+Numeric)` model performed worse than the `RandomForestClassifier (Text+Numeric)` (which used raw TF-IDF features), indicating that `TruncatedSVD` with `n_components=200` might have removed too much information or that the original TF-IDF features were more effective for this dataset.
*   Logistic Regression provided competitive results, particularly in cross-validation, but was slightly outperformed by the RandomForestClassifier with full text and numeric features.

### Insights or Next Steps
*   The `cleaned_combined_review` text feature is a significant contributor to model performance, as evidenced by the poor performance of the numeric-only model.
*   `TruncatedSVD` for dimensionality reduction on TF-IDF features did not improve performance in this case. Further tuning of `n_components` for TruncatedSVD or exploring other dimensionality reduction techniques could be considered, or simply sticking with the full TF-IDF features if computational resources allow.
*   The `RandomForestClassifier` appears to be a stronger model for this task compared to `Logistic Regression` when text features are included.
*   Further hyperparameter tuning for the `RandomForestClassifier` (e.g., `max_depth`, `min_samples_split`, `min_samples_leaf`) could potentially yield even better results.


## Final Task

### Subtask:
Summarize the findings and the performance of all models based on the newly updated comparison table, highlighting any changes due to the numeric features modification.


## Summary:

### Q&A
The analysis successfully summarized the findings and performance of all models by comparing their key metrics: Test Accuracy, Test F1-Macro, Test F1-Weighted, and Mean CV F1-Weighted. The comparison revealed that the `RandomForestClassifier (Text+Numeric)` model achieved the highest performance among the models using both text and numeric features. The `RandomForestClassifier (Numeric Only)` model showed the lowest performance, indicating that numeric features alone are not sufficient for good classification in this dataset.

### Data Analysis Key Findings
*   The `RandomForestClassifier (Text+Numeric)` model demonstrated the best overall performance, achieving the highest Test Accuracy of 0.6582, Test F1-Macro of 0.66, Test F1-Weighted of 0.66, and Mean CV F1-Weighted of 0.6245 among all evaluated models.
*   The `RandomForestClassifier (Numeric Only)` model performed poorly across all metrics (e.g., Test Accuracy: 0.3755, Test F1-Weighted: 0.38), underscoring the critical importance of text features for this classification task.
*   The `RandomForestClassifier (Text+SVD+Numeric)` model, incorporating `TruncatedSVD` for dimensionality reduction on text features, performed worse (Test Accuracy: 0.5527, Test F1-Weighted: 0.55) than the `RandomForestClassifier (Text+Numeric)` model, suggesting that `TruncatedSVD` with `n_components=200` might have removed crucial information or that raw TF-IDF features were more effective.
*   Logistic Regression (Text+Numeric) provided competitive results, particularly in cross-validation (Mean CV F1-Weighted: 0.6049), but was slightly outperformed by the best `RandomForestClassifier` configuration.

### Insights or Next Steps
*   The `cleaned_combined_review` text feature is a dominant predictor, as evidenced by the significantly lower performance of models relying solely on numeric features.
*   Further investigation into `TruncatedSVD` parameters (e.g., `n_components`) or exploring alternative dimensionality reduction techniques for text features could be beneficial, or simply sticking with the full TF-IDF features if computational resources allow.


In [ ]:
from sklearn.svm import LinearSVC

# Define the new model: LinearSVC
lsvm_model = LinearSVC(
    random_state=42,
    max_iter=1000,
    dual=False # 'dual=False' is recommended when n_samples > n_features, or when the problem is not well-conditioned
)

# Pipeline End-to-End with the existing preprocessor and LinearSVC
lsvm_pipeline = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
        ('classifier', lsvm_model)
    ]
)

print("LinearSVC model and pipeline defined.")

# Training Model
print("Training LinearSVC model...")
lsvm_pipeline.fit(X_train, y_train)
print("LinearSVC model training complete!")

# Evaluation on Test Set
print("\nEvaluating LinearSVC model on Test Set...")
# Prediksi
lsvm_y_pred = lsvm_pipeline.predict(X_test)

# Accuracy
lsvm_accuracy = accuracy_score(y_test, lsvm_y_pred)
print(f"LinearSVC Model Accuracy: {lsvm_accuracy:.4f}")

# Classification Report
print("\nLinearSVC Model Classification Report:")
print(classification_report(y_test, lsvm_y_pred))

# Confusion Matrix
print("LinearSVC Model Confusion Matrix:")
print(confusion_matrix(y_test, lsvm_y_pred))

# Cross-Validation (Stratified K-Fold)
print("\nPerforming Cross-Validation for LinearSVC Model...")
lsvm_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

lsvm_cv_scores = cross_val_score(
    lsvm_pipeline,
    X,
    y,
    cv=lsvm_cv,
    scoring='f1_weighted'
)

print("LinearSVC Model Cross-Validation F1-Weighted Scores:", lsvm_cv_scores)
print("LinearSVC Model Mean CV F1-Weighted:", lsvm_cv_scores.mean())


LinearSVC model and pipeline defined.
Training LinearSVC model...
LinearSVC model training complete!

Evaluating LinearSVC model on Test Set...
LinearSVC Model Accuracy: 0.6245

LinearSVC Model Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.59      0.61        80
           1       0.54      0.51      0.53        80
           2       0.68      0.78      0.73        77

    accuracy                           0.62       237
   macro avg       0.62      0.63      0.62       237
weighted avg       0.62      0.62      0.62       237

LinearSVC Model Confusion Matrix:
[[47 25  8]
 [19 41 20]
 [ 7 10 60]]

Performing Cross-Validation for LinearSVC Model...
LinearSVC Model Cross-Validation F1-Weighted Scores: [0.57909183 0.58162736 0.548295   0.63492427 0.602312  ]
LinearSVC Model Mean CV F1-Weighted: 0.5892500908598512


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

# Redefine numeric features to include both for this specific model
mnb_numeric_features = ['bayesian_score']

# Create a new preprocessor for Multinomial Naive Bayes
# TfidfVectorizer for text, MinMaxScaler for numeric features (to ensure non-negativity)
mnb_preprocessor = ColumnTransformer(
    transformers=[
        (
            'text',
            TfidfVectorizer(
                ngram_range=(1, 2),
                max_features=5000,
                min_df=5,
                stop_words='english'
            ),
            text_feature
        ),
        (
            'num',
            MinMaxScaler(), # Using MinMaxScaler to ensure non-negative numeric features
            mnb_numeric_features
        )
    ]
)

# Define the new model: Multinomial Naive Bayes
mnb_model = MultinomialNB()

# Pipeline End-to-End with the new preprocessor and MultinomialNB
mnb_pipeline = Pipeline(
    steps=[
        ('preprocessing', mnb_preprocessor),
        ('classifier', mnb_model)
    ]
)

print("Multinomial Naïve Bayes model and pipeline defined.")

# Training Model
print("Training Multinomial Naïve Bayes model...")
mnb_pipeline.fit(X_train, y_train)
print("Multinomial Naïve Bayes model training complete!")

# Evaluation on Test Set
print("\nEvaluating Multinomial Naïve Bayes model on Test Set...")
# Prediksi
mnb_y_pred = mnb_pipeline.predict(X_test)

# Accuracy
mnb_accuracy = accuracy_score(y_test, mnb_y_pred)
print(f"Multinomial Naïve Bayes Model Accuracy: {mnb_accuracy:.4f}")

# Classification Report
print("\nMultinomial Naïve Bayes Model Classification Report:")
print(classification_report(y_test, mnb_y_pred))

# Confusion Matrix
print("Multinomial Naïve Bayes Model Confusion Matrix:")
print(confusion_matrix(y_test, mnb_y_pred))

# Cross-Validation (Stratified K-Fold)
print("\nPerforming Cross-Validation for Multinomial Naïve Bayes Model...")
mnb_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

mnb_cv_scores = cross_val_score(
    mnb_pipeline,
    X,
    y,
    cv=mnb_cv,
    scoring='f1_weighted'
)

print("Multinomial Naïve Bayes Model Cross-Validation F1-Weighted Scores:", mnb_cv_scores)
print("Multinomial Naïve Bayes Model Mean CV F1-Weighted:", mnb_cv_scores.mean())


Multinomial Naïve Bayes model and pipeline defined.
Training Multinomial Naïve Bayes model...
Multinomial Naïve Bayes model training complete!

Evaluating Multinomial Naïve Bayes model on Test Set...
Multinomial Naïve Bayes Model Accuracy: 0.5612

Multinomial Naïve Bayes Model Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.35      0.48        80
           1       0.46      0.72      0.56        80
           2       0.64      0.61      0.62        77

    accuracy                           0.56       237
   macro avg       0.62      0.56      0.55       237
weighted avg       0.62      0.56      0.55       237

Multinomial Naïve Bayes Model Confusion Matrix:
[[28 41 11]
 [ 6 58 16]
 [ 3 27 47]]

Performing Cross-Validation for Multinomial Naïve Bayes Model...
Multinomial Naïve Bayes Model Cross-Validation F1-Weighted Scores: [0.54627286 0.50847511 0.5261321  0.48833127 0.53198105]
Multinomial Naïve Bayes Model Mean CV F1-We